In [1]:

import os

import geopandas as gpd
import pandas as pd
from numpy import mean

from numpy import std

import numpy as np
import re


In [3]:
points1 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/eckartsau_dgm_S1.gpkg' 

points1 = gpd.read_file(points1)
points1 = pd.DataFrame(points1.drop(columns='geometry'))


points2 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/Friesach_dgm_S1.gpkg' 

points2 = gpd.read_file(points2)
points2 = pd.DataFrame(points2.drop(columns='geometry'))


points3 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/Mattsee_dgm_S1.gpkg' 

points3 = gpd.read_file(points3)
points3 = pd.DataFrame(points3.drop(columns='geometry'))


points4 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/Mieming_dgm_S1.gpkg' 

points4 = gpd.read_file(points4)
points4 = pd.DataFrame(points4.drop(columns='geometry'))


points5 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/Ternitz_dgm_S1.gpkg' 

points5 = gpd.read_file(points5)
points5 = pd.DataFrame(points5.drop(columns='geometry'))


points6 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/Tobaj_dgm_S1.gpkg' 

points6 = gpd.read_file(points6)
points6 = pd.DataFrame(points6.drop(columns='geometry'))


points7 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/Villach_dgm_S1.gpkg' 

points7 = gpd.read_file(points7)
points7 = pd.DataFrame(points7.drop(columns='geometry'))


points8 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/Weitra_dgm_S1.gpkg' 

points8 = gpd.read_file(points8)
points8 = pd.DataFrame(points8.drop(columns='geometry'))


points9 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/sportplaetze_dgm_S1.gpkg' 

points9 = gpd.read_file(points9)
points9 = pd.DataFrame(points9.drop(columns='geometry'))


points10 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/gletscher_data_dgm_S1.gpkg' 

points10 = gpd.read_file(points10)
points10 = pd.DataFrame(points10.drop(columns='geometry'))

points11 = '/mnt/eo/projekt/2021_STATeo/01_Data/02_labelled_prepared/Punkte_stat2austria/neusiedler_see_dgm_S1.gpkg' 

points11 = gpd.read_file(points11)
points11 = pd.DataFrame(points11.drop(columns='geometry'))


In [4]:
df = pd.concat([points1, points2, points3, points4, points5, points5, points6, points7, points8, points9, points10, points11])
df = df.fillna(np.nan)

df['Response'] = df['Response'].replace('bare rock and screes', 'Bare rock and screes')

In [5]:
data = df
target = 'Response'
response = df['Response']
data = df.iloc[:, 0:(len(data.columns)-1)]
X = data
y = response


import dill
path_parent = os.path.dirname(os.getcwd())
dill.load_session('/mnt/eo/projekt/2021_STATeo/01_Data/Österreich/Model/Model_Oesterreich4.db')



In [6]:
# select groups
# gruppe 1 sentinel 2 bands
# gruppe 2 NDVI
# gruppe 3 NDWI
# gruppe 4 DGM_Slope

# gruppe 5 Sentinel 1

g1 = data.loc[:,[True if re.search('B+',columns) else False for columns in data.columns]]
g2 = data.loc[:,['NDVI' in i for i in data.columns]]
g3 = data.loc[:,['NDWI' in i for i in data.columns]]
g4 = data.loc[:,['DGM' in i for i in data.columns]]
g4_5 = data.loc[:,['SLOPE' in i for i in data.columns]]
g5 = data.loc[:,['S1' in i for i in data.columns]]

In [8]:
g1 = g1.reset_index()
g2 = g2.reset_index()
g3 = g3.reset_index()
g4 = g4.reset_index()
g4_5 = g4_5.reset_index()
g5 = g5.reset_index()

In [9]:
parameters = {
    "max_depth": [list_params_5['max_depth']],
    "learning_rate": [list_params_5['learning_rate']],
    "max_bins":[list_params_5['max_bins']],
    "max_iter":[list_params_5['max_iter']],
    "l2_regularization":[list_params_5['l2_regularization']]
}


In [10]:
final_model = HistGradientBoostingClassifier(max_depth = max_dpth, learning_rate = learner_rate,
 max_bins = mx_bins, max_iter = mx_iter, l2_regularization = regularization)


In [11]:
a = [g1, g2, g3, g4, g4_5, g5]
group_idx = list(range(0, len(a)))
group_idx

cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 25)
scores_development = []

variable_idxs = 0
best_score = 0
X = 0
Xs = 0
idx_development = []
start_length = len(group_idx)

while len(group_idx) > 0:
    best_score = 0
    for i in range(0, len(group_idx)):
        idx = group_idx[i]
        if len(group_idx) == start_length:
            Xtemp = a[idx]
        else:
            
            Xtemp = pd.concat([Xe, a[idx]], axis = 1, join = 'inner')

        n_scores = cross_val_score(final_model, y = response, X = Xtemp, scoring='balanced_accuracy', cv=cv, n_jobs=-1)
        mean_score = np.mean(n_scores)
        print(mean_score)
        if mean_score > best_score:
            variable_idxs = i
            best_score = mean_score
            best_idx = idx

    Xs = a[best_idx]
    if len(group_idx) == start_length:
        Xe = Xs
    else:
        
        Xe =  pd.concat([Xe, Xs], axis = 1, join = 'inner')
    
    group_idx.pop(variable_idxs)
    print(len(group_idx))
    scores_development.append(best_score)
    
    idx_development.append(best_idx)


0.8666113168011997
0.8441704879999254
0.838222971546555
0.6924417627732643
0.4506563274192606
0.7775065947613709
5
0.8665257097368031
0.8674982150641489
0.8833597849428145
0.8746003598546046
0.8706122081387998
4
0.8844514868772675
0.8837437931485937
0.8866689716955991
0.8849528575494416
3
0.8857171165177894
0.8866947994532574
0.8881017124597997
2
0.8876325697021595
0.888647033722647
1
0.8881745541914009
0


In [12]:
textfile = open("/mnt/eo/projekt/2021_STATeo/01_Data/score_dev_austria22.txt", "w")
for element in scores_development:
    textfile.write(str(element) + "\n")
textfile.close()